<a href="https://colab.research.google.com/github/SatadruMukherjee/customer_review_insight_extractor/blob/main/automated_insight_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anthropic boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [2]:
customer_reviews="""The food was outstanding! Each dish was a work of art both in taste and presentation.
We celebrated our anniversary here and the staff went above and beyond to make it special.
The menu offered a wide range of options and everything we tried was delicious.
The steak was cooked to perfection and the wine selection was excellent.
The view from the rooftop terrace is stunning and the food matches the scenery.
The sushi rolls were fresh and flavorful and the presentation was beautiful.
The best Italian food I've had in the city! The pasta was handmade and cooked perfectly.
The restaurant has a lovely atmosphere and the staff was friendly and knowledgeable.
The food was good but the noise level in the restaurant was a bit high making it hard to enjoy our meal.
The dessert was delicious but I wish there were more variety on the menu."""

In [3]:
prompt=f"""Human: You are a summarisation assistant. Your task is to summarise product reviews given to you as a list. Within this list, there are individual product reviews in an array.
Create a JSON document with the following fields:
summary - A summary of these reviews in less than 250 words
overall_sentiment - The overall sentiment of the reviews
sentiment_confidence - How confident you are about the sentiment of the reviews
reviews_positive - The percent of positive reviews
reviews_neutral - The percent of neutral reviews
reviews_negative - The percent of negative reviews
action_items - A list of action items to resolve the customer complaints (don't put soemthing which is already good and there is no customer complaints)
Your output should be raw JSON - do not include any sentences or additional text outside of the JSON object.
Here is the list of reviews that I want you to summarise:
{customer_reviews}
Assistant:"""

In [4]:
import anthropic
import boto3
from datetime import datetime

In [5]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="{}",
)
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
)


In [6]:
output = message.content

In [13]:
import json

In [14]:
analysis=json.loads(output[0].text)

In [15]:
analysis

{'summary': "The reviews overwhelmingly praise the restaurant for its outstanding food quality, presentation, and taste. Customers particularly enjoyed the steak, sushi, pasta, and desserts. The restaurant's atmosphere, including a rooftop terrace with stunning views, was highly appreciated. Staff were commended for their friendliness, knowledge, and exceptional service, especially for special occasions. The wine selection and menu variety were also positively noted. However, a few minor concerns were raised, including high noise levels in the restaurant and a desire for more dessert options.",
 'overall_sentiment': 'Highly Positive',
 'sentiment_confidence': 0.95,
 'reviews_positive': 90,
 'reviews_neutral': 10,
 'reviews_negative': 0,
 'action_items': ['Consider implementing noise reduction measures to improve dining experience',
  'Expand the dessert menu to offer more variety']}

In [16]:
dynamodb = boto3.resource('dynamodb',aws_access_key_id='{AWS_ACCESS_KEY_ID}',aws_secret_access_key='{AWS_SECRET_ACCESS_KEY}',region_name='us-east-1')
table = dynamodb.Table('review_storer_yt')
table.put_item(
    Item={
        'date': datetime.today().strftime('%Y-%m-%d'),
        'create_time': datetime.today().strftime('%Y-%m-%dT%H:%M:%S'),
        'reviews_summary': analysis["summary"],
        'overall_sentiment': analysis["overall_sentiment"],
        'sentiment_confidence': str(analysis["sentiment_confidence"]),
        'reviews_positive': str(analysis["reviews_positive"]),
        'reviews_negative': str(analysis["reviews_negative"]),
        'reviews_neutral': str(analysis["reviews_neutral"]),
        'action_items': ', '.join(analysis['action_items']),
    }
)

{'ResponseMetadata': {'RequestId': '13NGEMCPTMR8LN4K1AIQNF1N13VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 01 Sep 2024 07:53:44 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '13NGEMCPTMR8LN4K1AIQNF1N13VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}